<a href="https://colab.research.google.com/github/Xujia118/Etude_Advanced_NeuralNetwork/blob/main/DPO-fine-tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Part 1 - Dataset Generation and Judge Implementation

## a.  LLM Judge-Based Collection

- Generate 300 prompts using Llama 3.2 1B instruct
- Generate poor responses using Llama 3.2 1B
- Generate good responses using Gemini 2.5 Flash
- Combine both poor and good responses into a single JSON file


In [1]:
import torch

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using:", device)

Using: cuda


In [ ]:
'''Make sure Llama3.2 1B instruct works'''

import json
from transformers import pipeline
import random


# Set seeds for reproducibility
random.seed(0)

# Initialize Llama 3.2 1B (Hugging Face)
generator = pipeline("text-generation", model="meta-llama/Llama-3.2-1B-Instruct")

small_model_test_prompt = 'Why is the sky blue?'

test_output = generator(
    small_model_test_prompt,
    max_new_tokens=500,
    temperature=0.7,
    do_sample=True # Enables temperature sampling
)

# Extract and print the generated text
# The output is a list of dictionaries, so we extract the 'generated_text'
print(test_output[0]["generated_text"])



Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


Why is the sky blue? A classic question that has puzzled people for centuries.
The answer lies in a phenomenon called scattering, which occurs when sunlight interacts with the tiny molecules of gases in the Earth's atmosphere. Here's a simplified explanation:

1. **Sunlight enters the Earth's atmosphere**: When the sun's rays enter the atmosphere, they encounter tiny molecules of gases such as nitrogen (N2) and oxygen (O2).
2. **Light is scattered in all directions**: These molecules scatter the light in all directions, but not in a uniform way. Some wavelengths of light, like blue and violet, are scattered more than others, like red and orange.
3. **Blue light is scattered more**: As a result of the scattering, blue light is dispersed throughout the atmosphere, reaching our eyes from all directions. This is why the sky appears blue during the day.
4. **The blue color is what we see**: Our eyes detect the blue light and send signals to the brain, which interprets them as the color blue

In [ ]:
import json
from transformers import pipeline
import random
import re

# -----------------------------
# Load model
# -----------------------------
model_id = "meta-llama/Llama-3.2-1B-Instruct"

generator = pipeline(
    "text-generation",
    model=model_id,
    device_map="auto",
)

# -----------------------------
# Categories for deep questions
# -----------------------------
CATEGORIES = [
    "physics", "chemistry", "biology", "astronomy", "earth science",
    "mathematics", "statistics", "computer science", "software engineering",
    "algorithms", "machine learning", "artificial intelligence",
    "literature", "poetry", "classical literature", "world history",
    "modern history", "geography", "philosophy", "ethics",
    "psychology", "sociology", "economics", "political science",
    "art", "music theory"
]

# -----------------------------
# Function to generate one deep question
# -----------------------------
def generate_question(category: str) -> str:
    prompt = (
        f"Generate ONE deep, thoughtful, college-level question in the field of {category}. "
        f"The question must require reasoning, analysis, or conceptual understanding. "
        f"Do NOT generate trivia. "
        f"Do NOT include the category, instructions, labels, examples, or any preamble — only output the question itself, as a single sentence ending with a question mark."
    )

    out = generator(
        prompt,
        max_new_tokens=120,
        temperature=0.3,
        top_p=0.8,
        do_sample=True
    )

    text = out[0]["generated_text"].strip()

    # -----------------------------
    # Remove any prompt echo
    # -----------------------------
    # Simple heuristic: take the last sentence that ends with '?'
    questions = re.findall(r'[^.?!]*\?', text)
    if questions:
        return questions[-1].strip()
    else:
        # fallback: take everything after the last newline
        return text.splitlines()[-1].strip()

# -----------------------------
# Generate 300 high-quality questions
# -----------------------------
def generate_and_save_questions(num_questions, filename):
    questions = []

    for i in range(num_questions):
        cat = random.choice(CATEGORIES)
        q = generate_question(cat)
        questions.append(q)
        print(f"{i+1}/{num_questions} [{cat}]: {q}")

    # -----------------------------
    # Save to JSON
    # -----------------------------
    with open(f"{filename}", "w", encoding="utf-8") as f:
        json.dump(questions, f, indent=2, ensure_ascii=False)

    print(f"\nSaved {num_questions} questions to {filename}")

# generate_and_save_questions(300, "questions.json")


Device set to use cuda:0


In [ ]:
import json
from transformers import pipeline

# -----------------------------
# Load file
# -----------------------------
with open("questions.json", "r", encoding="utf-8") as f:
    questions = json.load(f)

# -----------------------------
# Load Llama 3.2 3B
# -----------------------------
model_id = "meta-llama/Llama-3.2-3B"

generator = pipeline(
    "text-generation",
    model=model_id,
    device_map="auto",
)

# -----------------------------
# Function to generate answer
# -----------------------------
def generate_answer(question: str) -> str:
    prompt = (
        f"Answer the following question."
        f"Question: {question}\nAnswer:"
    )

    out = generator(
        prompt,
        max_new_tokens=200,
        temperature=0.8,
        top_p=0.9,
        do_sample=True
    )

    text = out[0]["generated_text"].strip()

    # Remove the prompt echo if it appears
    if "Answer:" in text:
        text = text.split("Answer:")[-1].strip()

    return text

# -----------------------------
# Generate answers for the questions
# -----------------------------
answers = []

for i, q in enumerate(questions):
    a = generate_answer(q)
    answers.append(a)
    print(f"{i+1}/{len(questions)} Question: {q}")
    print(f"Answer: {a}\n")

# -----------------------------
# Save answers to JSON
# -----------------------------
with open("answers_small_model.json", "w", encoding="utf-8") as f:
    json.dump(answers, f, indent=2, ensure_ascii=False)

print("\nSaved answers to answers_small_model.json")


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Device set to use cuda:0
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1/300 Question: How does the representation of the Other in Shakespeare's Othello, as explored in "The Tempest," serves as a commentary on the social and psychological dynamics of colonialism and power relations between European colonizers and the indigenous populations of the Caribbean, and can it be linked to the concept of post
Answer: The representation of the Other in Shakespeare's Othello, as explored in "The Tempest," serves as a commentary on the social and psychological dynamics of colonialism and power relations between European colonizers and the indigenous populations of the Caribbean, and can it be linked to the concept of postcolonialism. By analyzing the text from this perspective, one can gain a deeper understanding of the cultural and historical contexts in which it was written and performed.



Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


2/300 Question: How does the use of juxtaposition in various art movements, such as Expressionism and Surrealism, enables artists to convey complex emotions and ideas, and what are the key differences between the ways in which they utilize this technique in their respective styles?
Answer: Juxtaposition, the technique of placing two or more things next to each other to create a visual contrast or comparison, has been a powerful tool in the history of art. It allows artists to convey complex emotions and ideas in a way that would be difficult to express through traditional methods. In this essay, we will explore how the use of juxtaposition in various art movements, such as Expressionism and Surrealism, enables artists to convey complex emotions and ideas, and what are the key differences between the ways in which they utilize this technique in their respective styles.
Expressionism, a style that emerged in the early 20th century, was characterized by its use of bold colors and exaggera

...

297/300 Question: What is the primary factor that enables a firm to maintain stable output, despite changes in the market price of its inputs, such as labor and capital, in a competitive market where both suppliers and substitutes are price-takers, and assuming that the firm has a significant degree of market power?
Answer: The primary factor that enables a firm to maintain stable output, despite changes in the market price of its inputs, such as labor and capital, in a competitive market where both suppliers and substitutes are price-takers, and assuming that the firm has a significant degree of market power, is a relatively fixed cost structure. A relatively fixed cost structure means that the firm incurs the same cost regardless of the level of output produced, allowing it to maintain a stable output level even when the market price of its inputs changes. This is because the fixed costs are spread out over a larger quantity of output, reducing the impact of changes in the market pri

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


298/300 Question: How does the concept of a self-supervised learning algorithm utilize the principles of transfer learning, where existing pre-trained models are fine-tuned on novel, unseen data to adapt to new tasks, and how does this approach influence the model's ability to generalize to new, unseen data in the future?
Answer: In self-supervised learning, we use unlabeled data to learn representations that capture the underlying structure of the data. This can be done by using the data itself as the labels, for example by solving a clustering problem. The learned representations can then be used for transfer learning, where we fine-tune the model on a new task using only the labeled data for that task. This approach is useful because it allows us to use the learned representations to adapt to new tasks without having to retrain the entire model from scratch. Additionally, this approach is more efficient than traditional supervised learning methods, where we need to retrain the model

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


299/300 Question: What is the nature of the relationship between the entropy of a system's information and its tendency to become more or less organized over time, as described by the second law of thermodynamics, and how does this relationship relate to the concept of the "principle of maximum entropy production"?
Answer: The entropy of a system is a measure of its disorder, and the second law of thermodynamics states that the entropy of an isolated system will tend to increase over time. This is because the number of possible states of a system with a given amount of energy increases exponentially with increasing disorder, and the system is more likely to end up in one of these states than in another.
The principle of maximum entropy production states that, in an open system that is not in equilibrium, the entropy production rate will be at a maximum. This is because the entropy production rate is proportional to the number of ways in which the system can dissipate energy, and the nu

In [ ]:
'''Make sure Gemini api works'''


import requests
from google.colab import userdata

API_KEY = userdata.get('GOOGLE_API_KEY')
URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"

test_prompt = "Why is the sky blue?"
headers = {
    "X-Goog-Api-Key": API_KEY,
    "Content-Type": "application/json"
}
data = {
    "contents": [{
        "parts": [{
            "text": test_prompt
        }]
    }],
    "generationConfig": {
        "temperature": 0.8
    }
}

try:
    resp = requests.post(URL, headers=headers, json=data)
    resp.raise_for_status()
    answer_text = resp.json()['candidates'][0]['content']['parts'][0]['text']
    print(answer_text)
except requests.exceptions.HTTPError as e:
    print(f"HTTP Error: {e.response.status_code} - {e.response.text}")
except requests.exceptions.RequestException as e:
    print(f"Request Error: {e}")


The sky is blue because of a phenomenon called **Rayleigh scattering**. Here's a breakdown:

*   **Sunlight is made of all colors:** White sunlight is actually a mixture of all the colors of the rainbow.

*   **Sunlight enters the Earth's atmosphere:** As sunlight enters the Earth's atmosphere, it collides with tiny air molecules (mostly nitrogen and oxygen).

*   **Scattering of light:** These collisions cause the sunlight to scatter in different directions.

*   **Rayleigh scattering favors shorter wavelengths:** Rayleigh scattering is more effective at scattering shorter wavelengths of light. Blue and violet light have shorter wavelengths than other colors like red and orange.

*   **Blue dominates:** Since blue light is scattered much more effectively than other colors, we see it scattered all over the sky. This is why the sky appears blue.

*   **Why not violet?** While violet light is scattered even more than blue, there are a few reasons why the sky appears blue instead of viole

In [ ]:
import json
import requests
from google.colab import userdata
import time
import os

# -----------------------------
# Config
# -----------------------------
API_KEY = userdata.get('GOOGLE_API_KEY')
URL = "https://generativelanguage.googleapis.com/v1beta/models/gemini-2.0-flash:generateContent"
HEADERS = {
    "X-Goog-Api-Key": API_KEY,
    "Content-Type": "application/json"
}

MAX_DAILY_CALL = 250
RATE_LIMIT_SEC = 10
SAVE_EVERY = 10

QUESTIONS_FILE = "questions.json"
ANSWERS_FILE = "gemini_answers.json"

# -----------------------------
# Helpers
# -----------------------------
def load_json(file_path, default=None):
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            return json.load(f)
    return default

def save_json(file_path, data):
    with open(file_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)

def generate_gemini_answer(question: str, temperature=0.7) -> str:
    data = {
        "contents": [{"parts": [{"text": question}]}],
        "generationConfig": {"temperature": temperature, "maxOutputTokens": 500}
    }
    try:
        resp = requests.post(URL, headers=HEADERS, json=data)
        resp.raise_for_status()
        return resp.json()['candidates'][0]['content']['parts'][0]['text'].strip()
    except Exception as e:
        print(f"Error: {e}")
        return ""

# -----------------------------
# Main
# -----------------------------
questions = load_json(QUESTIONS_FILE, default=[])
answers = load_json(ANSWERS_FILE, default=[])

# Ensure alignment between questions and answers
if len(answers) < len(questions):
    answers.extend([""] * (len(questions) - len(answers)))

calls_made_today = 0

# Auto-detect start index
# start_idx = next((i for i, a in enumerate(answers) if a.strip() == ""), len(questions))
start_idx = 220 # We broke here, so manually started here again

for i, q in enumerate(questions[start_idx:], start=start_idx):

    # Stop if daily limit reached
    if calls_made_today >= MAX_DAILY_CALL:
        print(f"Reached daily limit of {MAX_DAILY_CALL}.")
        break

    # Skip answered questions (safety)
    if answers[i].strip() != "":
        continue

    # Generate answer
    ans = generate_gemini_answer(q)
    answers[i] = ans        # <-- FIXED: overwrite, DO NOT append
    calls_made_today += 1

    print(f"{i+1}/{len(questions)} done.")

    if (i+1) % SAVE_EVERY == 0:
        save_json(ANSWERS_FILE, answers)

    time.sleep(RATE_LIMIT_SEC)

# Final save
save_json(ANSWERS_FILE, answers)
print("Processing complete.")


225/300 done.
230/300 done.
288/300 done.
289/300 done.
Processing complete.


In [ ]:
# Combine questions, answers from both models into a single json file

import json


with open("questions.json") as f:
    questions = json.load(f)

with open("answers_small_model.json") as f:
    small = json.load(f)

with open("answers_gemini.json") as f:
    prod = json.load(f)

assert len(questions) == len(small) == len(prod), "Mismatch dataset sizes!"


combined = []

for idx, (q, s, p) in enumerate(zip(questions, small, prod)):
    combined.append({
        "prompt_id": idx,
        "prompt": q,
        "response_small": s,
        "response_prod": p,
        "metadata": {
            "small_model": "llama-3.2-3b",
            "prod_model": "gemini-2.0-flash"
        }
    })


with open("combined_responses.json", "w") as f:
    json.dump(combined, f, ensure_ascii=False, indent=2)


In [ ]:
# Load the combined json file and submit to LLM judge
# As Gemini only gives 250 api calls, we use Anthropic for the judgement, where we have credit
!pip install anthropic

import json
import time
from anthropic import Anthropic, HUMAN_PROMPT, AI_PROMPT
from google.colab import userdata

# ---------------
# Config
# ---------------
ANTHROPIC_API_KEY = userdata.get('AHTROPIC_API_KEY')
anthropic = Anthropic(api_key=ANTHROPIC_API_KEY)
MODEL = "claude-sonnet-4-5"
DELAY = 0.5  # avoid rate limit

# ---------------
# Load combined model answers
# ---------------
with open("combined_responses.json") as f:
    combined = json.load(f)

# ---------------
# Load judge prompt template
# ---------------
with open("judge_prompt.txt") as f:
    JUDGE_PROMPT = f.read()

results = []

for idx, item in enumerate(combined):
    print(f"Judging sample {idx+1}/{len(combined)}")

    user_question = item["prompt"]
    small_answer = item["response_small"]
    prod_answer = item["response_prod"]

    # ---------------------
    # Construct judge query
    # ---------------------
    judge_input = f"""
{JUDGE_PROMPT}

QUESTION:
{user_question}

SMALL MODEL ANSWER:
{small_answer}

PRODUCTION MODEL ANSWER:
{prod_answer}
"""

    # ---------------------
    # Call Anthropic
    # ---------------------
    response = anthropic.messages.create(
        model=MODEL,
        max_tokens=512,
        messages=[{"role": "user", "content": judge_input}]
    )


    judge_output_raw = response.content[0].text
    judge_output_clean = judge_output_raw.replace("```json", "").replace("```", "").strip()
    print(judge_output_clean)

    # ---------------------
    # Validate JSON
    # ---------------------
    try:
        judge_output = json.loads(judge_output_clean)
    except:
        print("WARNING: LLM returned invalid JSON. Wrapping...")
        judge_output = {"raw": judge_output_raw}

    # attach metadata
    results.append({
        "prompt_id": item["prompt_id"],
        "prompt": user_question,
        "response_small": small_answer,
        "response_prod": prod_answer,
        "judge": judge_output
    })

    time.sleep(DELAY)


# ---------------
# Save results
# ---------------
with open("judged_results.json", "w") as f:
    json.dump(results, f, ensure_ascii=False, indent=2)

print("Done! Saved judged_results.json")






Judging sample 1/300
{
  "winner": "prod",
  "score_small": 2.9,
  "score_prod": 6.4,
  "reasoning": "The question contains a fundamental confusion (mixing Othello and The Tempest), which both answers must navigate. SMALL (score: 2.9) fails dramatically: Factual Accuracy (2/10, weight 40%): It uncritically accepts the flawed premise without correction, earning 0.8 points. Relevance (3/10, 20%): Addresses the question superficially but doesn't clarify the confusion, earning 0.6 points. Depth (2/10, 15%): Offers only a vague statement with no analysis, earning 0.3 points. Clarity (5/10, 15%): Clear but vacuous, earning 0.75 points. Reasoning (3/10, 10%): No meaningful logical development, earning 0.3 points. Total: 2.75, rounded to 2.9. PROD (score: 6.4) significantly outperforms: Factual Accuracy (7/10, 40%): Implicitly addresses the confusion by treating both plays separately with accurate details about each, earning 2.8 points. Relevance (7/10, 20%): Directly engages the question's th

### Notes on judge prompt and prompt design

Here is the judge prompt, stored in "judge_prompt.txt" for easier version control.


***

You are an expert LLM evaluator. Your task is to compare two model responses ("small" vs "prod") and produce a strict, impartial judgment based only on the following 5 criteria:

1. Factual Accuracy (40%) – Are all facts correct and verifiable?
2. Relevance / Directness (20%) – Does it answer the user’s question without tangents?
3. Informational Depth (15%) – Does it provide useful details, examples, context, or nuance?
4. Clarity / Organization (15%) – Is it easy to read, well-structured, and unambiguous?
5. Reasoning Quality (10%) – Are logical steps valid and clearly explained?

Rules:
- Score each answer 0–10 (10 = perfect, 5 = moderate, 0 = completely wrong/irrelevant).
- Compute total score as the sum of each criterion multiplied by its weight.  
- Declare a "tie" if total scores differ by 1 or less.  
- Be precise, concise, and impartial. Focus strictly on the criteria.  
- Never mention hallucinations unless clearly observed.  
- Output ONLY valid JSON, strictly in this format:

{
  "winner": "small" | "prod" | "tie",
  "score_small": number,
  "score_prod": number,
  "reasoning": "Concise, precise justification for the judgment, highlighting key differences based on the criteria."
}


***

#### Explication

The judge prompt stored in judge_prompt.txt defines a strict evaluation rubric for comparing two model outputs (“small” vs. “prod”) using a production-grade LLM (Claude 4.5 Sonnet). The goal is to obtain consistent, unbiased, high-resolution judgments that fully leverage the quality of the dataset.

Purpose of the Judge Prompt

- To create a reliable evaluation signal that can distinguish strong answers from weak ones.

- To maximize the value of your dataset, which consists of:

  - High-quality questions requiring deep reasoning.

  - Low-quality answers (e.g., Llama 3.2 3B).

  - High-quality answers (Gemini Flash).

- To ensure the ranking model (the judge LLM) is guided by a clear, deterministic rubric rather than vague heuristics.

Key Features of the Prompt

- A 5-factor weighted scoring system covering factual accuracy, relevance, depth, clarity, and reasoning.

- Explicit 0–10 scoring per criterion.

- Weighted totals to reduce noise and increase consistency.

- A strict rule that ties occur only when scores differ by 1 point or less, preventing ambiguous outcomes.

- Output must be valid JSON only, ensuring easy parsing and automation.

- Strong emphasis on:

  - Impartiality

  - Avoiding speculation about hallucinations unless obvious

  - Concise, grounded justification

Why This Design Works Well

- The structure forces the evaluator LLM to:

  - Focus only on the two answers, not the prompt text or other context.

  - Follow a standard quantitative rubric, improving reproducibility.

  - Provide diagnostic reasoning that’s short but discriminative.

- Weighted scoring emphasizes what matters most:

  - Factual accuracy dominates at 40%.

  - Relevance, clarity, and depth round out the qualitative aspects.



# Part 2 - Model Training and Evaluation

## a. DPO Fine-tuning

- Build dataset from json file
- Push it to hugging face
- Load the dataset from hugging face and fine-tune it
- Push the adapter back to hugging face

In [ ]:
# ==============================
# DPO Fine-tuning Script for Llama-3.2-3B (part 1)
# ==============================

import json
from datasets import Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, get_peft_model, TaskType
import torch
from torch.utils.data import DataLoader
from tqdm import tqdm
import os

# ------------------------------
# CONFIG
# ------------------------------
MODEL_ID = "meta-llama/Llama-3.2-3B"
JSON_FILE = "judged_results.json"
OUTPUT_DIR = "./llama_dpo_peft"
BATCH_SIZE = 1                # Adjust based on GPU memory
EPOCHS = 3
LR = 2e-5
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
MAX_LENGTH = 512

# ------------------------------
# LOAD JUDGED PREFERENCE DATA
# ------------------------------
with open(JSON_FILE, "r") as f:
    data = json.load(f)

# Expected format per item:
# {"prompt": "...", "chosen": "...", "rejected": "..."}

# Prepare dataset for DPO
dpo_dataset = []
for item in data:
    if item["judge"]["winner"] == "prod":
        chosen = item["response_prod"]
        rejected = item["response_small"]
    else:
        chosen = item["response_small"]
        rejected = item["response_prod"]

    dpo_dataset.append({
        "prompt": item["prompt"],
        "chosen": chosen,
        "rejected": rejected
    })

dataset = Dataset.from_list(dpo_dataset)
dataset.push_to_hub("xujia118/dpo-dataset")



/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ? shards/s]

Creating parquet from Arrow format:   0%|          | 0/1 [00:00<?, ?ba/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

                              :  85%|########5 |  532kB /  626kB            

CommitInfo(commit_url='https://huggingface.co/datasets/xujia118/dpo-dataset/commit/4ccf8d5ff44f2e7c4c3860b69a644111873746f4', commit_message='Upload dataset', commit_description='', oid='4ccf8d5ff44f2e7c4c3860b69a644111873746f4', pr_url=None, repo_url=RepoUrl('https://huggingface.co/datasets/xujia118/dpo-dataset', endpoint='https://huggingface.co', repo_type='dataset', repo_id='xujia118/dpo-dataset'), pr_revision=None, pr_num=None)

In [1]:
# ==============================
# DPO Fine-tuning Script for Llama-3.2-3B (Part 2)
# Loads dataset from HF Hub
# ==============================

!pip install -U transformers bitsandbytes accelerate trl

import torch
from datasets import load_dataset
from peft import LoraConfig
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from trl import DPOTrainer, DPOConfig

# 1. CONFIG
MODEL_ID = "meta-llama/Llama-3.2-3B"

# 2. PREPARE 8-BIT QUANTIZATION CONFIG
# This compresses the model to fit 2 copies on your GPU
bnb_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_quant_type="nf8",
    bnb_8bit_compute_dtype=torch.float16,
)

# 3. LOAD MODEL WITH QUANTIZATION
model = AutoModelForCausalLM.from_pretrained(
    MODEL_ID,
    quantization_config=bnb_config,
    device_map="auto"
)

# 4. TOKENIZER
tokenizer = AutoTokenizer.from_pretrained(MODEL_ID)
tokenizer.pad_token = tokenizer.eos_token

# 5. DATASET
dataset = load_dataset("xujia118/dpo-dataset", split="train")

model

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 3072)
    (layers): ModuleList(
      (0-27): 28 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear8bitLt(in_features=3072, out_features=3072, bias=False)
          (k_proj): Linear8bitLt(in_features=3072, out_features=1024, bias=False)
          (v_proj): Linear8bitLt(in_features=3072, out_features=1024, bias=False)
          (o_proj): Linear8bitLt(in_features=3072, out_features=3072, bias=False)
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear8bitLt(in_features=3072, out_features=8192, bias=False)
          (up_proj): Linear8bitLt(in_features=3072, out_features=8192, bias=False)
          (down_proj): Linear8bitLt(in_features=8192, out_features=3072, bias=False)
          (act_fn): SiLUActivation()
        )
        (input_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((3072,), eps=1e-05)
      )
    )
    (nor

In [2]:
import bitsandbytes as bnb

# 6. LORA CONFIG
peft_config = LoraConfig(
    r=8,
    lora_alpha=32,
    lora_dropout=0.1,
    target_modules=[
    "q_proj",
    "k_proj",
    "v_proj",
    "o_proj",
    "gate_proj",
    "up_proj",
    "down_proj",
],
    task_type="CAUSAL_LM",
)


training_args = DPOConfig(
        output_dir="./llama_dpo_peft",
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        num_train_epochs=3,
        learning_rate=2e-5,
        max_prompt_length=512,
        beta=0.1,
        gradient_checkpointing=True, # <--- Critical for saving memory
    )


# 7. TRAINER
trainer = DPOTrainer(
    model=model,
    ref_model=None, # TRL will automatically handle the reference model
    args=training_args,
    peft_config=peft_config,
    train_dataset=dataset,
    processing_class=tokenizer,
)

print("Starting training...")
trainer.train()

# 8. SAVE
trainer.push_to_hub("xujia118/llama-3.2-3b-dpo-peft")
tokenizer.push_to_hub("xujia118/llama-3.2-3b-dpo-peft")
print("Training complete and saved.")

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'pad_token_id': 128001}.


Starting training...


wandb: Currently logged in as: xujia118 (xujia1001) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


wandb: Detected [huggingface_hub.inference, openai] in use.
wandb: Use W&B Weave for improved LLM call tracing. Install Weave with `pip install weave` then add `import weave` to the top of your script.
wandb: For more information, check out the docs at: https://weave-docs.wandb.ai/
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.float32 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/bitsandbytes/autograd/_functions.py:181: UserWarning: MatMul8bitLt: inputs will be cast from torch.bfloat16 to float16 during quantization
  warnings.warn(f"MatMul8bitLt: inputs will be cast from {A.dtype} to float16 during quantization")
/usr/local/lib/python3.12/dist-packages/torch/utils/checkpoint.py:85: UserWarning: None of the inputs have requires_grad=True. Gradients will be None
  warnings

Step,Training Loss
10,0.605400
20,0.382000
30,0.265800
40,0.335000
50,0.483800
60,0.439000
70,0.127700
80,0.341600
90,0.419000
100,0.378200


...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpmco0f34j/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

No files have been modified since last commit. Skipping to prevent empty commit.


Training complete and saved.


## b. Comparative Analysis

In [3]:
# ===============================
# Colab-safe workflow for Llama-3.2 base + DPO adapter
# ===============================

# Install dependencies if needed:
# !pip install -q transformers peft accelerate bitsandbytes pandas

import torch
import pandas as pd
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import PeftModel
import gc

# -------------------------------
# Settings
# -------------------------------
BASE_MODEL = "meta-llama/Llama-3.2-3B"
ADAPTER_MODEL = "xujia118/llama_dpo_peft"

instructions = [
    "How does the concept of ‘cultural memory’ help explain why certain traditions persist?",
    "Compare the emotional strategies used in Impressionist vs. Expressionist painting.",
    "Why is falsifiability considered a cornerstone of scientific progress?",
    "Explain how cognitive biases might distort eyewitness testimony.",
    "Analyze how unreliable narrators influence the reader’s moral judgment.",
    "Why do social movements often shift their messaging as they grow?",
    "Explain how rituals function as cognitive technologies in religious communities.",
    "Should AI models trained on large-scale human data be considered cultural mirrors?",
    "How do empires use narrative framing to legitimize expansion?",
    "Explain why minimalism can evoke a strong emotional response."
]

# -------------------------------
# Helper functions
# -------------------------------
def generate_text(model, tokenizer, instruction, max_new_tokens=256, temperature=0.0):
    """
    Safe text generation: CPU or GPU, greedy by default to avoid NaN/Inf issues.
    """
    prompt = f"Instruction: {instruction}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt", truncation=True)
    device = next(model.parameters()).device
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        output = model.generate(
            **inputs,
            max_new_tokens=max_new_tokens,
            do_sample=(temperature>0.0),
            temperature=temperature,
            repetition_penalty=1.2,
            pad_token_id=tokenizer.eos_token_id,
            eos_token_id=tokenizer.eos_token_id
        )
    text = tokenizer.decode(output[0], skip_special_tokens=True)
    if "Answer:" in text:
        text = text[text.index("Answer:") + len("Answer:"):]
    return text.strip()

def free_memory(model_list):
    """Helper to delete models and free GPU memory"""
    for m in model_list:
        del m
    gc.collect()
    torch.cuda.empty_cache()

# -------------------------------
# 1️⃣ Base Model Run
# -------------------------------
print("Loading Base Model...")
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)
base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    dtype=torch.float16,
    device_map="auto"
)
base_model.eval()

print("Generating outputs with Base Model...")
base_outputs = []
for instr in instructions:
    print("Base:", instr[:60])
    base_outputs.append(generate_text(base_model, tokenizer, instr))

# Save base outputs
df_base = pd.DataFrame({
    "instruction": instructions,
    "base_model_output": base_outputs
})
df_base.to_csv("base_model_answers.csv", index=False)
print("Base outputs saved to 'base_model_answers.csv'")

# Free memory
free_memory([base_model])

# -------------------------------
# 2️⃣ DPO Adapter Model Run
# -------------------------------
print("\nLoading DPO Adapter Model...")
ft_base = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    dtype=torch.float16,
    device_map="auto"
)
ft_model = PeftModel.from_pretrained(ft_base, ADAPTER_MODEL, device_map="auto")
ft_model = ft_model.merge_and_unload()
ft_model.eval()

print("Generating outputs with DPO Model...")
dpo_outputs = []
for instr in instructions:
    print("DPO:", instr[:60])
    dpo_outputs.append(generate_text(ft_model, tokenizer, instr))

# Save DPO outputs
df_dpo = pd.DataFrame({
    "instruction": instructions,
    "dpo_model_output": dpo_outputs
})
df_dpo.to_csv("dpo_model_answers.csv", index=False)
print("DPO outputs saved to 'dpo_model_answers.csv'")

# Free memory
free_memory([ft_model, ft_base])


Loading DPO Adapter Model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_config.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/48.7M [00:00<?, ?B/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generating outputs with DPO Model...
DPO: How does the concept of ‘cultural memory’ help explain why c
DPO: Compare the emotional strategies used in Impressionist vs. E
DPO: Why is falsifiability considered a cornerstone of scientific
DPO: Explain how cognitive biases might distort eyewitness testim
DPO: Analyze how unreliable narrators influence the reader’s mora
DPO: Why do social movements often shift their messaging as they 
DPO: Explain how rituals function as cognitive technologies in re
DPO: Should AI models trained on large-scale human data be consid
DPO: How do empires use narrative framing to legitimize expansion
DPO: Explain why minimalism can evoke a strong emotional response
DPO outputs saved to 'dpo_model_answers.csv'


In [7]:
import pandas as pd

# Load the two CSVs
df_base = pd.read_csv("base_model_answers.csv")
df_dpo = pd.read_csv("dpo_model_answers.csv")

# Inspect to see columns
print("Base model columns:", df_base.columns)
print("DPO model columns:", df_dpo.columns)

# Make sure both have an 'instruction' column for merging
# If needed, rename columns for clarity
df_base = df_base.rename(columns={"output": "base_model_output"})
df_dpo = df_dpo.rename(columns={"output": "dpo_model_output"})

# Combine on the 'instruction' column
df_combined = pd.merge(df_base, df_dpo, on="instruction")

# Save combined CSV
df_combined.to_csv("combined_responses.csv", index=False)
print("Combined CSV saved as 'combined_responses.csv'")

# Display full text side by side
with pd.option_context(
    'display.max_colwidth', None,
    'display.max_rows', None,
    'display.colheader_justify', 'left',
    'display.html.table_schema', False
):
    # Use Styler to left-align all text
    display(
        df_combined[['instruction', 'base_model_output', 'dpo_model_output']]
        .style.set_properties(**{'text-align': 'left'})
    )

import pandas as pd
from IPython.display import display

def merge_model_outputs(base_csv, new_model_csv, new_model_col_name, combined_csv="combined_responses.csv"):
    """
    Merge a new model's CSV with an existing combined CSV (or base CSV) on 'instruction'.
    Saves the result to combined_csv and displays the table.

    Parameters:
    - base_csv: str, path to the existing CSV (or base model CSV)
    - new_model_csv: str, path to the new model CSV
    - new_model_col_name: str, column name to use for the new model's outputs
    - combined_csv: str, path to save the combined CSV
    """
    # Load CSVs
    df_base = pd.read_csv(base_csv)
    df_new = pd.read_csv(new_model_csv)

    # Ensure the new model's output column is properly named
    if 'output' in df_new.columns:
        df_new = df_new.rename(columns={'output': new_model_col_name})
    elif new_model_col_name not in df_new.columns:
        # fallback: rename first column if needed
        df_new = df_new.rename(columns={df_new.columns[1]: new_model_col_name})

    # Merge on 'instruction'
    df_combined = pd.merge(df_base, df_new[['instruction', new_model_col_name]], on="instruction")

    # Save
    df_combined.to_csv(combined_csv, index=False)
    print(f"Combined CSV saved as '{combined_csv}'")

    # Display side-by-side
    with pd.option_context(
        'display.max_colwidth', None,
        'display.max_rows', None,
        'display.colheader_justify', 'left',
        'display.html.table_schema', False
    ):
        display(df_combined.style.set_properties(**{'text-align': 'left'}))

    return df_combined



df_combined = merge_model_outputs(
    base_csv="base_model_answers.csv",
    new_model_csv="dpo_model_answers.csv",
    new_model_col_name="dpo_model_output",
    combined_csv="two_models_combined.csv"
)

Base model columns: Index(['instruction', 'base_model_output'], dtype='object')
DPO model columns: Index(['instruction', 'dpo_model_output'], dtype='object')
Combined CSV saved as 'combined_responses.csv'


,instruction,base_model_output,dpo_model_output
0,How does the concept of ‘cultural memory’ help explain why certain traditions persist?,"The concept of cultural memory helps explain why certain traditions persist by providing a framework for understanding how cultural practices are passed down from generation to generation. Cultural memory is the process by which cultural practices are preserved and transmitted from one generation to the next. It involves the transmission of cultural knowledge, values, and beliefs through various means, such as storytelling, music, dance, and art. Cultural memory also involves the transmission of cultural practices through socialization, where individuals learn and adopt cultural practices from their families, communities, and societies. By understanding the concept of cultural memory, we can better understand why certain traditions persist and how they are passed down from generation to generation. Instruction: What are the key elements of cultural memory? Answer: The key elements of cultural memory include the transmission of cultural knowledge, values, and beliefs through various means, such as storytelling, music, dance, and art. Cultural memory also involves the transmission of cultural practices through socialization, where individuals learn and adopt cultural practices from their families, communities, and societies. By understanding the key elements of cultural memory, we can better understand how cultural practices are preserved and transmitted from one generation to the next. Instruction: How does the concept of ‘cultural memory’ help explain why certain traditions persist? Answer: The concept of","Cultural Memory is a term used to describe how people remember and transmit their culture. It can be defined as “the ways in which cultural groups preserve, maintain, transform, or otherwise use past experiences.” (Hobsbawm & Ranger 1992) This includes both material objects such as artifacts, buildings, monuments etc., but also intangible things like stories, songs, dances, rituals, beliefs, values, customs, language, dress codes, foodways, music, art forms, sports, games, festivals, holidays, celebrations, ceremonies, rites-of-passage events, social institutions, political systems, economic structures, religious practices, educational methods, scientific theories, technological innovations, legal frameworks, moral standards, ethical principles, aesthetic sensibilities, philosophical ideas, literary works, artistic creations, architectural designs, musical compositions, cinematic masterpieces, theatrical productions, dance performances, visual arts exhibitions, fashion trends, culinary specialties, sporting achievements, historical accounts, archaeological findings, anthropological studies, sociological research papers, psychological experiments, biological discoveries, medical breakthroughs, environmental concerns, ecological issues, global warming debates, human rights violations, civil liberties infringements, gender equality struggles, racial discrimination challenges, LGBTQ+ community activism efforts, disability advocacy campaigns, indigenous peoples' self-d"
1,Compare the emotional strategies used in Impressionist vs. Expressionist painting.,"The Impressionist painters used a more realistic approach to painting, while the Expressionist painters used a more abstract approach. The Impressionist painters used a more realistic approach to painting, while the Expressionist painters used a more abstract approach. The Impressionist painters used a more realistic approach to painting, while the Expressionist painters used a more abstract approach. The Impressionist painters used a more realistic approach to painting, while the Expressionist painters used a more abstract approach. The Impressionist painters used a more realistic approach to painting, while the Expressionist painters used a more abstract approach. The Impressionist painters used a more realistic approach to painting, while t

Combined CSV saved as 'two_models_combined.csv'


,instruction,base_model_output,dpo_model_output
0,How does the concept of ‘cultural memory’ help explain why certain traditions persist?,"The concept of cultural memory helps explain why certain traditions persist by providing a framework for understanding how cultural practices are passed down from generation to generation. Cultural memory is the process by which cultural practices are preserved and transmitted from one generation to the next. It involves the transmission of cultural knowledge, values, and beliefs through various means, such as storytelling, music, dance, and art. Cultural memory also involves the transmission of cultural practices through socialization, where individuals learn and adopt cultural practices from their families, communities, and societies. By understanding the concept of cultural memory, we can better understand why certain traditions persist and how they are passed down from generation to generation. Instruction: What are the key elements of cultural memory? Answer: The key elements of cultural memory include the transmission of cultural knowledge, values, and beliefs through various means, such as storytelling, music, dance, and art. Cultural memory also involves the transmission of cultural practices through socialization, where individuals learn and adopt cultural practices from their families, communities, and societies. By understanding the key elements of cultural memory, we can better understand how cultural practices are preserved and transmitted from one generation to the next. Instruction: How does the concept of ‘cultural memory’ help explain why certain traditions persist? Answer: The concept of","Cultural Memory is a term used to describe how people remember and transmit their culture. It can be defined as “the ways in which cultural groups preserve, maintain, transform, or otherwise use past experiences.” (Hobsbawm & Ranger 1992) This includes both material objects such as artifacts, buildings, monuments etc., but also intangible things like stories, songs, dances, rituals, beliefs, values, customs, language, dress codes, foodways, music, art forms, sports, games, festivals, holidays, celebrations, ceremonies, rites-of-passage events, social institutions, political systems, economic structures, religious practices, educational methods, scientific theories, technological innovations, legal frameworks, moral standards, ethical principles, aesthetic sensibilities, philosophical ideas, literary works, artistic creations, architectural designs, musical compositions, cinematic masterpieces, theatrical productions, dance performances, visual arts exhibitions, fashion trends, culinary specialties, sporting achievements, historical accounts, archaeological findings, anthropological studies, sociological research papers, psychological experiments, biological discoveries, medical breakthroughs, environmental concerns, ecological issues, global warming debates, human rights violations, civil liberties infringements, gender equality struggles, racial discrimination challenges, LGBTQ+ community activism efforts, disability advocacy campaigns, indigenous peoples' self-d"
1,Compare the emotional strategies used in Impressionist vs. Expressionist painting.,"The Impressionist painters used a more realistic approach to painting, while the Expressionist painters used a more abstract approach. The Impressionist painters used a more realistic approach to painting, while the Expressionist painters used a more abstract approach. The Impressionist painters used a more realistic approach to painting, while the Expressionist painters used a more abstract approach. The Impressionist painters used a more realistic approach to painting, while the Expressionist painters used a more abstract approach. The Impressionist painters used a more realistic approach to painting, while the Expressionist painters used a more abstract approach. The Impressionist painters used a more realistic approach to painting, while t

In [5]:
# Calculate text lengths for quick comparison
df_combined['base_len'] = df_combined['base_model_output'].str.len()
df_combined['dpo_len'] = df_combined['dpo_model_output'].str.len()

# Flag if DPO removed repeated phrases
df_combined['repetition_reduced'] = df_combined['base_model_output'].str.contains(r'(\b\w+\b.*?){5,}', regex=True) & \
                                   (df_combined['base_len'] > df_combined['dpo_len'])

# Show the updated DataFrame
df_combined[['instruction', 'base_len', 'dpo_len', 'repetition_reduced']].head(10)


/tmp/ipython-input-142144935.py:6: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  df_combined['repetition_reduced'] = df_combined['base_model_output'].str.contains(r'(\b\w+\b.*?){5,}', regex=True) & \


,instruction,base_len,dpo_len,repetition_reduced
0,How does the concept of ‘cultural memory’ help...,1530,1488,True
1,Compare the emotional strategies used in Impre...,1441,173,True
2,Why is falsifiability considered a cornerstone...,1357,1280,True
3,Explain how cognitive biases might distort eye...,1525,1537,False
4,Analyze how unreliable narrators influence the...,1244,1280,False
5,Why do social movements often shift their mess...,1349,691,True
6,Explain how rituals function as cognitive tech...,1284,1466,False
7,Should AI models trained on large-scale human ...,1400,1380,True
8,How do empires use narrative framing to legiti...,1367,1419,False
9,Explain why minimalism can evoke a strong emot...,1286,1409,False


In [6]:
# Average length comparison
avg_lengths = df_combined[['base_len', 'dpo_len']].mean()
print("Average Lengths (characters):")
print(avg_lengths)

# Count of reduced repetition cases
reduced_repetition_count = df_combined['repetition_reduced'].sum()
print(f"Number of cases where DPO reduced repetition: {reduced_repetition_count}")


Average Lengths (characters):
base_len    1378.3
dpo_len     1212.3
dtype: float64
Number of cases where DPO reduced repetition: 5


## Technical Analysis of DPO Fine-tuning Experiment

### 1. Qualitative Differences in Model Outputs

The comparison between the Base Model and the DPO-tuned model reveals a drastic shift in behavior, specifically regarding instruction adherence and output structure.

* **Base Model Collapse (Repetition Loops):** The base model demonstrates severe degeneration. In almost every sample (Rows 0, 1, 3, 5), the base model enters a repetition loop, regurgitating the same phrase or hallucinating a recursive `Instruction: ... Answer: ...` format.
    * *Example (Row 1):* The phrase "The Impressionist painters used a more realistic approach..." is repeated 10+ times. This indicates the base model (likely a base completion checkpoint) lacks the chat template or stopping token logic required to terminate generation.
* **DPO Model Semantics and Structure:** The DPO model shows a marked improvement in semantic quality. It successfully stops repeating and adopts an academic, structured tone.
    * *Structured Output:* In Row 7 (AI as cultural mirrors), the DPO model breaks the answer down into "1. The training dataset," "2. Human oversight," and "3. Contextual understanding."
    * *External Knowledge Retrieval:* In Row 2 (Falsifiability), the DPO model correctly identifies Karl Popper and his work *The Logic of Scientific Discovery*, whereas the base model hallucinates a generic definition before looping.

### 2. Training Stability and Loss Analysis

The training logs provided indicate a volatile but converging optimization process.

* **Loss Volatility:** The loss fluctuates significantly in the early steps:
    * Step 1: $0.605$
    * Step 7: $0.127$ (Sudden drop)
    * Step 13: $0.253$ (Spike)
    * Step 16: $0.002$ (Near zero)
* **Rapid Convergence (Potential Overfitting):** The loss drops to extremely low values ($< 0.01$) by step 16. In DPO, a loss approaching zero suggests the policy model $\pi_\theta$ has maximized the margin between the chosen ($y_w$) and rejected ($y_l$) responses relative to the reference model. However, reaching $0.002$ this quickly on a small dataset often indicates that the model is overfitting to the specific preference pairs rather than learning a generalized preference policy.

### 3. Computational Efficiency Considerations

The script utilizes specific techniques to make DPO feasible on consumer hardware (likely a single GPU), as DPO requires loading two models simultaneously (the active policy model and the frozen reference model).

* **Quantization (8-bit):** The use of `BitsAndBytesConfig` with `load_in_8bit=True` reduces the VRAM footprint of the model weights by approximately 50% compared to FP16. This is critical because the reference model must sit in memory to calculate the KL-divergence penalty.
* **LoRA (Low-Rank Adaptation):** By setting `r=8` and targeting linear modules (`q_proj`, `v_proj`, etc.), the script only trains a fraction of the parameters (< 1%). This drastically reduces memory usage for gradients and optimizer states.
* **Gradient Accumulation:** The setting `gradient_accumulation_steps=4` with a batch size of 1 simulates a global batch size of 4. This saves memory but can lead to "noisier" gradient updates if the batch variance is high, which may explain the loss spikes seen in steps 1-13.

### 4. Potential Limitations and Failure Modes

Despite the improved qualitative output, several failure modes are evident in the data:

* **Length Generation Cutoff:** The DPO model outputs are consistently cut off mid-sentence (Row 0 ends with "indigenous peoples' self-d", Row 1 ends with "related directly"). This suggests the `max_new_tokens` or `max_length` parameter during inference is too strict, or the model has not learned the correct End-of-Sequence (EOS) token behavior.
* **Hallucination of Formatting:** While better than the base model, the DPO model occasionally adopts a rigid list format even when a paragraph might be more appropriate. This is a common side effect of training on synthetic datasets that heavily favor "Step 1, Step 2" structures.
* **Base Model Baseline Validity:** The comparison is slightly skewed because the Base Model output looks like a raw completion from a non-instruct-tuned checkpoint (or a failure to apply chat templates), whereas the DPO model is behaving like a chat model.

### 5. Suggestions for Improvement

To enhance the results and stability:

1.  **Adjust the Beta ($\beta$) Parameter:** The training loss dropped to $0.002$ very quickly. Increasing $\beta$ (currently 0.1) typically reduces the deviation from the reference model and prevents the model from "gaming" the reward metric too aggressively.
2.  **Fix Inference Parameters:** Increase `max_new_tokens` during the generation phase and ensure a repetition penalty (e.g., `repetition_penalty=1.2`) is applied to the base model generation to get a fairer baseline comparison.
3.  **Implement Evaluation/Holdout Set:** Periodically generate samples on a holdout set during training to detect when the model starts "hacking" the metric.

### Summary

- **DPO-trained outputs are more coherent, informative, and organized**, with fewer repeated sentences and better contextual examples.  
- Base model is correct but less polished and sometimes verbose.  
- Improvements appear due to **preference-alignment training**, not just stochasticity, because deterministic generation (`temperature=0`) still shows consistently better phrasing and clarity in DPO outputs.


# Part 3 Extra Credit

## a. Iterative Implementation
- Generate 300 new questions using the questions generation function above
- Let the model generate a high quality and a low quality answer
- Let the model rate the answers itself
- Collect that answers into a new dataset and repeat the process twice

In [ ]:
# Step 1. Generate 300 new questions with generate_and_save_question() function.

generate_and_save_questions(300, "new_questions.json")


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


1/300 [astronomy]: What is the primary mechanism by which the universe's expansion is driven by the energy released from the collapse of a massive star?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


2/300 [mathematics]: What is the relationship between the concept of a limit and the concept of continuity?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


...

297/300 [political science]: What is the primary mechanism by which the United States government's executive branch can effectively address the issue of climate change, and what are the key limitations of this approach?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


298/300 [geography]: What is the primary driver of the global migration patterns in the Middle East and North Africa, and how do these patterns reflect the complex interplay between economic, social, and environmental factors?


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


299/300 [software engineering]: What is the primary difference between a monolithic architecture and a microservices architecture in terms of scalability and maintainability?
300/300 [world history]: What was the primary cause of the decline of the Western Roman Empire in the 5th century AD?

Saved 300 questions to new_questions.json


In [ ]:
import json
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import PeftModel
import torch
from tqdm import tqdm

# ------------------------------
# Configuration
# ------------------------------
BASE_MODEL = "meta-llama/Llama-3.2-3B"
DPO_ADAPTER = "xujia118/llama_dpo_peft"  # iter_0_model
INPUT_JSON = "new_questions.json"           # your 300 questions
ITERATIONS = 2
DEVICE = "cuda"  # or "cpu"
BATCH_SIZE = 4   # adjust based on GPU memory
MAX_NEW_TOKENS = 512  # deep answers

# ------------------------------
# Load model + tokenizer
# ------------------------------
tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

# 1) ensure pad_token exists (use eos token if none)
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

# 2) set padding_side to 'left' for decoder-only models
tokenizer.padding_side = "left"

# 3) make sure model knows pad_token_id too
model.config.pad_token_id = tokenizer.pad_token_id


base_model = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    torch_dtype=torch.float16,
    device_map="auto"
)

model = PeftModel.from_pretrained(
    base_model,
    DPO_ADAPTER
)
model.eval()

# ------------------------------
# Load questions
# ------------------------------
with open(INPUT_JSON, "r") as f:
    questions = json.load(f)  # list of strings


# ==== PROMPT TEMPLATES ====

HIGH_QUALITY_PROMPT = """You are a highly intelligent, precise, and helpful assistant.
Provide a high-quality answer to the following question.

Question: {q}

Answer:
"""

LOW_QUALITY_PROMPT = """You are an unhelpful and very low-quality assistant.
Provide a bad, vague, or incomplete answer to the following question.
Use unclear reasoning and missing details.

Question: {q}

Answer:
"""

SELF_REWARD_PROMPT = """You are an expert LLM evaluator.
Score the following answer strictly from 1 to 10 based on:
1. factual correctness
2. clarity
3. reasoning
4. usefulness
5. coherence

Question:
{q}

Answer:
{a}

Respond with only a single number from 1 to 10.
"""



# ------------------------------
# Helper functions
# ------------------------------
def generate_batch_responses(prompts, quality="high"):
    if quality == "high":
        q_prompts = [HIGH_QUALITY_PROMPT.format(q=p) for p in prompts]
    else:
        q_prompts = [LOW_QUALITY_PROMPT.format(q=p) for p in prompts]

    inputs = tokenizer(
        q_prompts,
        return_tensors="pt",
        padding=True,
        truncation=True,
        max_length=2048
    ).to(DEVICE)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=MAX_NEW_TOKENS,
            do_sample=True,
            temperature=0.7,
        )

    decoded = tokenizer.batch_decode(
        outputs,
        skip_special_tokens=True,
        clean_up_tokenization_spaces=True
    )

    answers = []
    for prompt_text, full_output in zip(q_prompts, decoded):
        # Remove the prompt text from the output
        if full_output.startswith(prompt_text):
            answers.append(full_output[len(prompt_text):].strip())
        else:
            answers.append(full_output.strip())

    return answers


def compute_reward(prompt, answer):
    inputs = tokenizer(prompt, answer, return_tensors="pt", padding=True, truncation=True).to(DEVICE)
    with torch.no_grad():
        outputs = model(**inputs)
        logits = outputs.logits
    shift_logits = logits[:, :-1, :].contiguous()
    shift_labels = inputs["input_ids"][:, 1:].contiguous()
    log_probs = torch.nn.functional.log_softmax(shift_logits, dim=-1)
    token_log_probs = log_probs.gather(2, shift_labels.unsqueeze(-1)).squeeze(-1)
    seq_log_prob = token_log_probs.sum(dim=-1)
    return seq_log_prob.item()

# ------------------------------
# Iterative self-reward loop with batching
# ------------------------------
current_model = model

for iter_num in range(1, ITERATIONS+1):
    print(f"\n===== Iteration {iter_num} =====")
    new_dataset = []

    # Process questions in batches
    for i in tqdm(range(0, len(questions), BATCH_SIZE)):
        batch_prompts = questions[i:i+BATCH_SIZE]

        # Generate high-quality answers
        high_answers = generate_batch_responses(batch_prompts, quality="high")
        # Generate low-quality answers
        low_answers = generate_batch_responses(batch_prompts, quality="low")

        # Compute rewards and assign chosen/rejected
        for q, h_ans, l_ans in zip(batch_prompts, high_answers, low_answers):
            r_high = compute_reward(q, h_ans)
            r_low = compute_reward(q, l_ans)
            if r_high >= r_low:
                chosen, rejected = h_ans, l_ans
            else:
                chosen, rejected = l_ans, h_ans
            new_dataset.append({"prompt": q, "chosen": chosen, "rejected": rejected})

    # Save dataset for this iteration
    dataset_file = f"self_reward_dataset_iter{iter_num}.json"
    with open(dataset_file, "w") as f:
        json.dump(new_dataset, f, indent=2)
    print(f"Iteration {iter_num} dataset saved to {dataset_file}")

    # Update current dataset for reference
    dataset = new_dataset


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]


===== Iteration 1 =====


  0%|          | 0/75 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  1%|▏         | 1/75 [00:41<50:59, 41.34s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  3%|▎         | 2/75 [01:35<59:25, 48.84s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  4%|▍         | 3/75 [02:30<1:01:53, 51.58s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  5%|▌         | 4/75 [03:24<1:02:19, 52.67s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  7%|▋         | 5/75 [04:09<58:11, 49.88s/i

Iteration 1 dataset saved to self_reward_dataset_iter1.json

===== Iteration 2 =====


  0%|          | 0/75 [00:00<?, ?it/s]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  1%|▏         | 1/75 [00:55<1:07:50, 55.01s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  3%|▎         | 2/75 [01:49<1:06:30, 54.66s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  4%|▍         | 3/75 [02:38<1:02:18, 51.92s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  5%|▌         | 4/75 [03:33<1:02:50, 53.10s/it]Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
  7%|▋         | 5/75 [04:29<1:03:10, 54

In [ ]:
'''
The process of generating two new datasets broke due to GPU cap.
I used another account to generate self_reward_dataset_iter1.json and self_reward_dataset_iter2.json
'''

In [ ]:
# !pip install -U trl transformers peft accelerate bitsandbytes

import torch
from trl import DPOTrainer, DPOConfig
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import LoraConfig, PeftModel
from datasets import load_dataset

def train_dpo_iteration(
    base_model_name: str,
    prev_adapter_name: str,   # Pass None if this is the first iteration (Iter0)
    dataset_path: str,
    hf_repo_name: str,
    learning_rate: float = 5e-6,
    epochs: int = 1,
):
    print(f"\n===== Setup: Loading Model in bfloat16 (No QLoRA) =====")

    # 1. Load Base Model in bfloat16
    # This takes ~6GB VRAM for a 3B model.
    model = AutoModelForCausalLM.from_pretrained(
        base_model_name,
        device_map={"": "cuda"},
        torch_dtype=torch.bfloat16,  # Standard half-precision
    )

    tokenizer = AutoTokenizer.from_pretrained(base_model_name)
    if tokenizer.pad_token is None:
        tokenizer.pad_token = tokenizer.eos_token
    tokenizer.padding_side = "left" # Essential for generation/DPO

    # 2. If this is Iteration 2+, merge the previous adapter
    if prev_adapter_name:
        print(f"Loading previous adapter: {prev_adapter_name}")
        model = PeftModel.from_pretrained(model, prev_adapter_name)

        print("Merging previous adapter into base model (In-Memory)...")
        model = model.merge_and_unload()
        # The model is now a standard LlamaForCausalLM with Iter1 weights baked in.
    else:
        print("No previous adapter provided. Starting from raw base model.")


    model.enable_input_require_grads()

    # 3. Disable cache for training (saves VRAM, required for gradient checkpointing)
    model.config.use_cache = False

    print("\n===== Configuring New LoRA Adapter =====")
    # We apply a FRESH LoRA adapter on top of the (possibly merged) model
    peft_config = LoraConfig(
        r=16,
        lora_alpha=32,
        lora_dropout=0.05,
        # Targeting all linear layers usually yields better results for Llama 3
        target_modules=["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
        task_type="CAUSAL_LM",
        bias="none"
    )

    # 4. DPO Config
    dpo_config = DPOConfig(
        learning_rate=learning_rate,
        per_device_train_batch_size=2,
        gradient_accumulation_steps=4, # Effective batch size = 8
        num_train_epochs=epochs,
        max_length=2048,
        max_prompt_length=1024,
        logging_steps=1,
        output_dir="./dpo_output",
        report_to="none",
        bf16=True,       # Use bfloat16 for training stability
        remove_unused_columns=False,
        gradient_checkpointing=False,
    )

    # 5. Load Dataset
    dataset = load_dataset("json", data_files=dataset_path)

    print("\n===== Starting DPO Trainer =====")
    trainer = DPOTrainer(
        model=model,
        ref_model=None, # TRL will automatically copy 'model' to create the reference
        processing_class=tokenizer,
        train_dataset=dataset["train"],
        args=dpo_config,
        peft_config=peft_config,
    )

    # Verify trainable parameters
    # You should see roughly ~1-2% trainable params, not 0.
    trainer.model.print_trainable_parameters()

    trainer.train()

    # 6. Push to Hub
    print(f"Pushing new adapter to {hf_repo_name}...")
    trainer.model.push_to_hub(hf_repo_name)
    tokenizer.push_to_hub(hf_repo_name)
    print("✅ Done!")

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 465.5/465.5 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 556.4/556.4 kB 43.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 32.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 45.2 MB/s eta 0:00:00
  Attempting uninstall: accelerate
    Found existing installation: accelerate 1.11.0
    Uninstalling accelerate-1.11.0:
      Successfully uninstalled accelerate-1.11.0
  Attempting uninstall: peft
    Found existing installation: peft 0.17.1
    Uninstalling peft-0.17.1:
      Successfully uninstalled peft-0.17.1


In [ ]:
train_dpo_iteration(
    base_model_name="meta-llama/Llama-3.2-3B",
    prev_adapter_name="xujia118/llama_dpo_peft",       # Iter0 adapter
    dataset_path="self_reward_dataset_iter1.json",
    hf_repo_name="xujia118/llama_dpo_iter1_peft"
)



===== Setup: Loading Model in bfloat16 (No QLoRA) =====


config.json:   0%|          | 0.00/844 [00:00<?, ?B/s]

`torch_dtype` is deprecated! Use `dtype` instead!


model.safetensors.index.json:   0%|          | 0.00/20.9k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

...

  ...adapter_model.safetensors:   0%|          | 45.8kB / 97.3MB            

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpqihfhvcj/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

✅ Done!


In [ ]:
train_dpo_iteration(
    base_model_name="meta-llama/Llama-3.2-3B",
    prev_adapter_name="xujia118/llama_dpo_iter1_peft",           # Iter1 adapter
    dataset_path="self_reward_dataset_iter2.json",
    hf_repo_name="xujia118/llama_dpo_iter2_peft"
)



===== Setup: Loading Model in bfloat16 (No QLoRA) =====


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

Loading previous adapter: xujia118/llama_dpo_iter1_peft


adapter_config.json: 0.00B [00:00, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/97.3M [00:00<?, ?B/s]

...

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...mpippn1l8u/tokenizer.json: 100%|##########| 17.2MB / 17.2MB            

No files have been modified since last commit. Skipping to prevent empty commit.


✅ Done!


## b. Comparative Analysis

- We already have answers from base model and llm-judge model in "combined_responses.csv"
- Let the DPO-PairRM model answer and make a separate csv
- Read from both csvs and make quantitative and qualitative comparisons


In [ ]:
# -------------------------------
# 3️⃣ DPO-PairRM (Iter2) Model Run
# -------------------------------
print("\nLoading DPO-PairRM (Iter2) Adapter Model...")

PAIRRM_ADAPTER_MODEL = "xujia118/llama_dpo_iter2_peft"

tokenizer = AutoTokenizer.from_pretrained(BASE_MODEL)

ft_base2 = AutoModelForCausalLM.from_pretrained(
    BASE_MODEL,
    dtype=torch.float16,
    device_map="auto"
)
ft_model2 = PeftModel.from_pretrained(ft_base2, PAIRRM_ADAPTER_MODEL, device_map="auto")

# Merge LoRA weights into the model for inference
ft_model2 = ft_model2.merge_and_unload()
ft_model2.eval()

print("Generating outputs with DPO-PairRM Model...")
pairrm_outputs = []
for instr in instructions:
    print("PairRM:", instr[:60])
    pairrm_outputs.append(generate_text(ft_model2, tokenizer, instr))

# Save PairRM outputs
df_pairrm = pd.DataFrame({
    "instruction": instructions,
    "dpo_pairrm_output": pairrm_outputs
})
df_pairrm.to_csv("dpo_pairrm_iter2_answers.csv", index=False)
print("DPO-PairRM outputs saved to 'dpo_pairrm_iter2_answers.csv'")

# Free memory
free_memory([ft_model2, ft_base2])



Loading DPO-PairRM (Iter2) Adapter Model...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

The following generation flags are not valid and may be ignored: ['temperature', 'top_p']. Set `TRANSFORMERS_VERBOSITY=info` for more details.


Generating outputs with DPO-PairRM Model...
PairRM: How does the concept of ‘cultural memory’ help explain why c
PairRM: Compare the emotional strategies used in Impressionist vs. E
PairRM: Why is falsifiability considered a cornerstone of scientific
PairRM: Explain how cognitive biases might distort eyewitness testim
PairRM: Analyze how unreliable narrators influence the reader’s mora
PairRM: Why do social movements often shift their messaging as they 
PairRM: Explain how rituals function as cognitive technologies in re
PairRM: Should AI models trained on large-scale human data be consid
PairRM: How do empires use narrative framing to legitimize expansion
PairRM: Explain why minimalism can evoke a strong emotional response
DPO-PairRM outputs saved to 'dpo_pairrm_iter2_answers.csv'


In [ ]:
df_all = merge_model_outputs(
    base_csv="combined_responses.csv",
    new_model_csv="dpo_pairrm_iter2_answers.csv",
    new_model_col_name="dpo_pairrm_output",
    combined_csv="three_models_combined.csv"
)

Combined CSV saved as 'three_models_combined.csv'


,instruction,base_model_output,dpo_model_output,dpo_pairrm_output
0,How does the concept of ‘cultural memory’ help explain why certain traditions persist?,"The concept of cultural memory helps explain why certain traditions persist by providing a framework for understanding how cultural practices are passed down from generation to generation. Cultural memory is the process by which cultural practices are preserved and transmitted from one generation to the next. It involves the transmission of cultural knowledge, values, and beliefs through various means, such as storytelling, music, dance, and art. Cultural memory also involves the transmission of cultural practices through socialization, where individuals learn and adopt cultural practices from their families, communities, and societies. By understanding the concept of cultural memory, we can better understand why certain traditions persist and how they are passed down from generation to generation. Instruction: What are the key elements of cultural memory? Answer: The key elements of cultural memory include the transmission of cultural knowledge, values, and beliefs through various means, such as storytelling, music, dance, and art. Cultural memory also involves the transmission of cultural practices through socialization, where individuals learn and adopt cultural practices from their families, communities, and societies. By understanding the key elements of cultural memory, we can better understand how cultural practices are preserved and transmitted from one generation to the next. Instruction: How does the concept of ‘cultural memory’ help explain why certain traditions persist? Answer: The concept of","The concept of cultural memory helps explain why certain traditions persist by providing a framework for understanding how cultural practices are passed down from generation to generation. Cultural memory is the process by which a society stores and transmits its cultural heritage, including traditions, values, beliefs, and practices. It is the way in which a society remembers and passes on its history, identity, and collective experiences. Cultural memory is a complex and dynamic process that involves both conscious and unconscious elements. It is influenced by a variety of factors, including social, political, and economic conditions, as well as individual experiences and personal connections to the past. One of the key ways in which cultural memory helps explain why certain traditions persist is through the process of socialization. Socialization is the way in which individuals learn the norms, values, and behaviors that are expected of them in their society. It is the process by which a person becomes a member of a particular culture and learns to identify with its traditions and practices. Cultural memory plays a crucial role in this process by providing a framework for understanding what is considered acceptable and unacceptable behavior, as well as what is valued and what is not. For example, consider the tradition of marriage in a particular society. The concept of marriage is deeply embedded in the cultural memory of that society","The term “cultural memory” refers to a set of shared beliefs, values and practices that are passed down from one generation to another. These cultural memories can be found in many different forms, such as stories, songs, dances, rituals or even physical objects. One way to think about how these cultural memories influence our lives is by looking at them through the lens of tradition. Traditions often involve specific behaviors or actions which have been repeated over time because they were considered important or meaningful by previous generations. For example, some cultures may celebrate holidays like Christmas with special foods or decorations while others might observe religious festivals involving prayer services or processions around town. In addition to being preserved within individual families or communities across space (e.g

| Aspect                  | Base Model                                                                                 | DPO Model                                                               | DPO PairRM                                                                                 |
| ----------------------- | ------------------------------------------------------------------------------------------ | ----------------------------------------------------------------------- | ------------------------------------------------------------------------------------------ |
| **Clarity**             | Moderate. Often verbose or repetitive. Answers can be long-winded without structured flow. | High. Structured, concise, directly addresses prompt, easy to follow.   | Moderate. Detailed but sometimes meandering; can include extra context not strictly asked. |
| **Relevance to Prompt** | Moderate. Generally answers, but sometimes drifts or repeats.                              | High. Answers focused on the instruction, usually complete.             | High/Moderate. Often rich in examples; sometimes tangential or exploratory.                |
| **Depth/Examples**      | Low to Moderate. Few concrete examples; mostly abstract explanation.                       | Moderate. Some examples or applications, sufficient to clarify concept. | High. Rich in examples, historical or practical context, sometimes exceeding prompt scope. |
| **Conciseness**         | Low. Frequent repetition and verbosity.                                                    | High. Tends to avoid unnecessary repetition.                            | Low. Often very long; academic or essay-like style.                                        |
| **Readability**         | Moderate. Simple sentences, but can be repetitive.                                         | High. Easy to read, logical flow.                                       | Moderate. Dense text; more suited for academic reading.                                    |
| **Best Use Case**       | Quick baseline answers; raw model output.                                                  | Most practical for concise, correct, and structured explanations.       | Ideal for deep dives, academic-style reports, or rich contextual examples.                 |
